In [1]:
from pathlib import Path
from io_f import read_data_file
from compute_f import split_ts_seq, compute_step_positions
import numpy as np
from pprint import pprint
from typing import Dict, List
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import os
import pandas as pd
from visualize_f import visualize_trajectory

In [2]:
def calibrate_magnetic_wifi_ibeacon_to_position(path_file_list):
    mwi_datas = {}
    for path_filename in path_file_list:
        # print(f'Processing {path_filename}...')

        path_datas = read_data_file(path_filename)
        acce_datas = path_datas.acce
        magn_datas = path_datas.magn
        ahrs_datas = path_datas.ahrs
        wifi_datas = path_datas.wifi
        ibeacon_datas = path_datas.ibeacon
        posi_datas = path_datas.waypoint

        step_positions = compute_step_positions(acce_datas, ahrs_datas, posi_datas)
        # visualize_trajectory(posi_datas[:, 1:3], floor_plan_filename, width_meter, height_meter, title='Ground Truth', show=True)
        # visualize_trajectory(step_positions[:, 1:3], floor_plan_filename, width_meter, height_meter, title='Step Position', show=True)
        # print(f"posi_datas[:,1:3]: {posi_datas[:,1:3]}")
        # print(f"step_positions[:,1:3]: {step_positions[:,1:3]}")
    
        if wifi_datas.size != 0:
            sep_tss = np.unique(wifi_datas[:, 0].astype(float))
            wifi_datas_list = split_ts_seq(wifi_datas, sep_tss)
            for wifi_ds in wifi_datas_list:
                diff = np.abs(step_positions[:, 0] - float(wifi_ds[0, 0]))
                index = np.argmin(diff)
                target_xy_key = tuple(step_positions[index, 1:3])   # indexing is exclusive it is step_positions[index,1] and step_positions[index,2]
                if target_xy_key in mwi_datas:
                    mwi_datas[target_xy_key]['wifi'] = np.append(mwi_datas[target_xy_key]['wifi'], wifi_ds, axis=0)
                else:
                    mwi_datas[target_xy_key] = {
                        'magnetic': np.zeros((0, 4)),
                        'wifi': wifi_ds,
                        'ibeacon': np.zeros((0, 3))
                    }

        if ibeacon_datas.size != 0:
            sep_tss = np.unique(ibeacon_datas[:, 0].astype(float))
            ibeacon_datas_list = split_ts_seq(ibeacon_datas, sep_tss)
            for ibeacon_ds in ibeacon_datas_list:
                diff = np.abs(step_positions[:, 0] - float(ibeacon_ds[0, 0]))
                index = np.argmin(diff)
                target_xy_key = tuple(step_positions[index, 1:3])
                if target_xy_key in mwi_datas:
                    mwi_datas[target_xy_key]['ibeacon'] = np.append(mwi_datas[target_xy_key]['ibeacon'], ibeacon_ds, axis=0)
                else:
                    mwi_datas[target_xy_key] = {
                        'magnetic': np.zeros((0, 4)),
                        'wifi': np.zeros((0, 5)),
                        'ibeacon': ibeacon_ds
                    }

        sep_tss = np.unique(magn_datas[:, 0].astype(float))
        magn_datas_list = split_ts_seq(magn_datas, sep_tss)
        for magn_ds in magn_datas_list:
            diff = np.abs(step_positions[:, 0] - float(magn_ds[0, 0]))
            index = np.argmin(diff)
            target_xy_key = tuple(step_positions[index, 1:3])
            if target_xy_key in mwi_datas:
                mwi_datas[target_xy_key]['magnetic'] = np.append(mwi_datas[target_xy_key]['magnetic'], magn_ds, axis=0)
            else:
                mwi_datas[target_xy_key] = {
                    'magnetic': magn_ds,
                    'wifi': np.zeros((0, 5)),
                    'ibeacon': np.zeros((0, 3))
                }

    return mwi_datas



In [3]:
# Loading all site 1 files
site_1_floors = ['B1','F1','F2','F3','F4']
site_2_floors = ['B1','F1','F2','F3','F4','F5','F6','F7','F8']

site_1_mwi_data: Dict = {floor:None for floor in site_1_floors}      # load mwi data for each floor
site_2_mwi_data: Dict = {floor:None for floor in site_2_floors}      # load mwi data for each floor

# Load site 1 data
print("Loading data for site 1.")
for floor in site_1_floors:
    floor_data_dir = f'./data/site1/{floor}/path_data_files'
    
    path_filenames = list(Path(floor_data_dir).resolve().glob("*.txt"))
    
    # get mwi data for each floor
    mwi_data = calibrate_magnetic_wifi_ibeacon_to_position(path_filenames)
    site_1_mwi_data[floor] = mwi_data
    print(f"{floor} done. {len(path_filenames)} files for {floor} loaded.")
    
    
# load Site 2 data
print("Loading data for site 2.")
for floor in site_2_floors:
    floor_data_dir = f'./data/site2/{floor}/path_data_files'
    
    path_filenames = list(Path(floor_data_dir).resolve().glob("*.txt"))
    
    # get mwi data for each floor
    mwi_data = calibrate_magnetic_wifi_ibeacon_to_position(path_filenames)
    site_2_mwi_data[floor] = mwi_data
    print(f"{floor} done. {len(path_filenames)} files for {floor} loaded.")
    

Loading data for site 1.
B1 done. 160 files for B1 loaded.
F1 done. 120 files for F1 loaded.
F2 done. 123 files for F2 loaded.
F3 done. 117 files for F3 loaded.
F4 done. 122 files for F4 loaded.
Loading data for site 2.
B1 done. 70 files for B1 loaded.
F1 done. 99 files for F1 loaded.
F2 done. 45 files for F2 loaded.
F3 done. 40 files for F3 loaded.
F4 done. 27 files for F4 loaded.
F5 done. 44 files for F5 loaded.
F6 done. 67 files for F6 loaded.
F7 done. 33 files for F7 loaded.
F8 done. 28 files for F8 loaded.


In [7]:
def magnetic_similarity_matrix(X_test, X_train, k=5):
    """
    Optimized magnetic similarity matrix with top-k mean.
    """
    T = X_test.shape[0]
    J = X_train.shape[0]
    similarities = np.zeros((T, J), dtype=np.float32)
    
    # Precompute masks for test sequences
    mask_test = ~np.isnan(X_test[..., 0])
    
    for i in range(T):
        # Get test sequence i
        Xi = X_test[i]
        mask_i = mask_test[i]
        valid_i = Xi[mask_i]
    
        # Normalize valid test vectors
        norm_i = np.linalg.norm(valid_i, axis=1, keepdims=True)
        valid_i_norm = valid_i / np.where(norm_i > 0, norm_i, 1.0)
        
        for j in range(J):
            # Get training sequence j
            Xj = X_train[j]
            mask_j = ~np.isnan(Xj[:, 0])
            valid_j = Xj[mask_j]
                
            # Normalize valid training vectors
            norm_j = np.linalg.norm(valid_j, axis=1, keepdims=True)
            valid_j_norm = valid_j / np.where(norm_j > 0, norm_j, 1.0)
            
            # Compute pairwise cosine similarities
            S = valid_i_norm @ valid_j_norm.T
            
            # Apply top-k mean
            flat_S = S.ravel()
            k_val = min(k, flat_S.size)
            # Get top-k values using partition for efficiency
            topk = np.partition(flat_S, -k_val)[-k_val:]
            similarities[i, j] = np.mean(topk)
    
    return similarities
def top_k_pred(similarities, Y_train, K=5):
    """
    Compute predictions using the weighted average of top-K similar training points.
    
    Args:
        similarities: np.ndarray of shape (n_test, n_train), higher = more similar
        Y_train: np.ndarray of shape (n_train, 2), coordinates of training points
        K: int, number of top similarities to consider
    
    Returns:
        Y_pred: np.ndarray of shape (n_test, 2)
    """
    n_test = similarities.shape[0]
    Y_pred = np.zeros((n_test, 2), dtype=float)
    
    # --- 1. Get top-K indices for each test sample ---
    top_k_idx = np.argpartition(-similarities, K-1, axis=1)[:, :K]
    
    # --- 2. Compute the weights for each top-K neighbor ---
    # Get the top K similarities for each test sample
    top_k_similarities = np.take_along_axis(similarities, top_k_idx, axis=1)  # shape: (n_test, K)
    
    # Normalize the similarities so they sum to 1 (to treat them as probabilities/weights)
    top_k_weights = top_k_similarities / np.sum(top_k_similarities, axis=1, keepdims=True)  # shape: (n_test, K)
    
    # --- 3. Compute weighted average of coordinates ---
    # Use advanced indexing to get all top-K coordinates at once
    top_k_coords = Y_train[top_k_idx]  # shape: (n_test, K, 2)
    
    # Compute weighted average along axis=1 (the K dimension)
    Y_pred = np.sum(top_k_coords * top_k_weights[..., np.newaxis], axis=1)  # shape: (n_test, 2)
    
    return Y_pred

def magnetic_floor_lookup(floor_data, test_size, random_state, k):
    """
    Perform magnetic-only table lookup localization for a single floor using top-K median.
    
    Args:
        floor_data: dict
            Dictionary mapping (x, y) coordinates to data, where each value has a "magnetic" array.
        test_size: float
            Fraction of data to use as test set.
        random_state: int
            Random seed for train/test splitting.
        k: int
            Number of top similar training points to consider for computing the median prediction.
    
    Returns:
        mean_dev: float
            Mean deviation (Euclidean) between predicted and true positions.
        median_dev: float
            Median deviation (Euclidean) between predicted and true positions.
    """

    # --- 1. Collect lengths of magnetic readings for all positions ---
    N_list = [v["magnetic"].shape[0] for v in floor_data.values()]
    Nmax = max(N_list)
    Nmin = min(N_list)
    Nmedian = np.median(N_list)
    Nmean = np.mean(N_list)

    # print(f"Magnetic readings per position -> "
    #       f"min: {Nmin}, mean: {Nmean:.2f}, median: {Nmedian}, max: {Nmax}")

    num_samples = len(floor_data)
    X = np.full((num_samples, Nmax, 3), np.nan)
    Y = np.zeros((num_samples, 2), dtype=float)

    # Collect all values for global normalization
    all_x, all_y, all_z = [], [], []

    for idx, (coord, data) in enumerate(floor_data.items()):
        Y[idx, 0], Y[idx, 1] = coord
        mag = data["magnetic"][:, 1:4]  # skip timestamp → use x,y,z
        N = mag.shape[0]
        X[idx, :N, :] = mag
        all_x.extend(mag[:, 0])
        all_y.extend(mag[:, 1])
        all_z.extend(mag[:, 2])

    # --- 2. Global normalization ---
    mu = np.array([np.mean(all_x), np.mean(all_y), np.mean(all_z)])
    sigma = np.array([np.std(all_x), np.std(all_y), np.std(all_z)])
    X = (X - mu) / sigma
    X = X.astype(np.float32)
    
    # --- 3. Train/test split ---
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )
    
    # --- 4. Compute similarity matrix ---
    similarities = magnetic_similarity_matrix(X_test, X_train, k)

    # --- 5. Predict using nearest neighbor ---
    Y_pred = top_k_pred(similarities, Y_train, K=k)

    # --- 6. Compute deviations ---
    deviations = np.sqrt(np.sum((Y_pred - Y_test) ** 2, axis=1))
    mean_dev = np.mean(deviations)
    median_dev = np.median(deviations)

    return mean_dev, median_dev

def run_magnetic_experiment(site_name, site_data, test_sizes, random_states, k_values):
    """Run experiment for magnetic floor lookup with different k values"""
    floors = list(site_data.keys())
    
    results = []
    
    # Iterate through all combinations
    for floor in floors:
        print(f"Processing floor {floor}...")
        floor_data = site_data[floor]
        
        for ts in test_sizes:
            for rs in random_states:
                for k in k_values:
                    mean_dev, median_dev = magnetic_floor_lookup(floor_data, ts, rs, k)
                    results.append((floor, ts, rs, k, mean_dev, median_dev))
                    print(f"  Test size: {ts}, Random state: {rs}, K: {k} "
                          f"-> Mean Dev: {mean_dev:.2f}, Median Dev: {median_dev:.2f}")
    
    # Create DataFrame with all results
    df = pd.DataFrame(
        results,
        columns=['Floor', 'Test_Size', 'Random_State', 'K', 'Mean_Dev', 'Median_Dev']
    )
    
    # Compute averages for each K, averaging over random_states only
    avg_results = df.groupby(['Floor', 'Test_Size', 'K']).agg({
        'Mean_Dev': 'mean',    # average mean deviation over random states
        'Median_Dev': 'mean'   # average median deviation over random states
    }).reset_index()

    # Round the columns to 4 decimals for clean CSV
    avg_results = avg_results.round(4)
    
    # Flatten column names properly
    avg_results.columns = ['Floor', 'Test_Size', 'K', 'Mean_Dev_Mean','Median_Dev_Mean']
    
    print(f"Results for {site_name}:")
    print(avg_results.to_string(index=False))
    print("\n" + "="*80 + "\n")
    
    return df, avg_results


# Main execution block
if __name__ == "__main__":
    # Experiment parameters
    test_sizes = [0.1]
    random_states = [0]
    k_values = [1, 5]  # Different k values to test
    
    # Assuming you have site_1_mwi_data and site_2_mwi_data
    # Run experiment for site 1
    print("Starting experiment for Site 1...")
    df1, pivot1 = run_magnetic_experiment(
        "Site 1", 
        site_1_mwi_data, 
        test_sizes, 
        random_states, 
        k_values
    )
    
    # Run experiment for site 2
    print("Starting experiment for Site 2...")
    df2, pivot2 = run_magnetic_experiment(
        "Site 2", 
        site_2_mwi_data, 
        test_sizes, 
        random_states, 
        k_values
    )
    
        # Round all numeric columns to 4 decimal places
    df1 = df1.round(4)
    df2 = df2.round(4)
    pivot1 = pivot1.round(4)
    pivot2 = pivot2.round(4)

    # Save results to CSV files
    df1.to_csv("site1_magnetic_topk_results.csv", index=False)
    df2.to_csv("site2_magnetic_topk_results.csv", index=False)
    pivot1.to_csv("site1_magnetic_topk_summary.csv", index=False)
    pivot2.to_csv("site2_magnetic_topk_summary.csv", index=False)

    print("Experiments completed!")

Starting experiment for Site 1...
Processing floor B1...
  Test size: 0.1, Random state: 0, K: 1 -> Mean Dev: 22.88, Median Dev: 2.86
  Test size: 0.1, Random state: 0, K: 5 -> Mean Dev: 50.45, Median Dev: 46.91
Processing floor F1...
  Test size: 0.1, Random state: 0, K: 1 -> Mean Dev: 19.83, Median Dev: 4.01
  Test size: 0.1, Random state: 0, K: 5 -> Mean Dev: 37.53, Median Dev: 34.64
Processing floor F2...
  Test size: 0.1, Random state: 0, K: 1 -> Mean Dev: 26.20, Median Dev: 7.02
  Test size: 0.1, Random state: 0, K: 5 -> Mean Dev: 43.11, Median Dev: 40.42
Processing floor F3...
  Test size: 0.1, Random state: 0, K: 1 -> Mean Dev: 25.35, Median Dev: 10.20
  Test size: 0.1, Random state: 0, K: 5 -> Mean Dev: 42.91, Median Dev: 38.71
Processing floor F4...
  Test size: 0.1, Random state: 0, K: 1 -> Mean Dev: 23.29, Median Dev: 4.31
  Test size: 0.1, Random state: 0, K: 5 -> Mean Dev: 42.31, Median Dev: 39.05
Results for Site 1:
Floor  Test_Size  K  Mean_Dev_Mean  Median_Dev_Mean
   